# Final report

### 3 Nodes Graph

Following the paper "A QAOA solution to the traveling salesman problem using pyQuil" by Matthew Radzihovsky, Joey Murphy, Mason Swofford, we encode the problem into an n^2 bitstring (or equivalently into an nxn matrix), being n the number of nodes in the grpah (n = 3 in our case).
Rows of the matrix can be interpreted as nodes in the graph, columns as time steps. Each row and each column must have exactly one 1 and n-1 zeros (meaning that 1 and only 1 city is visited at each time step).

We then encode the minimization problem + the hard constraints into a hamiltonian acting on nxn qubits. 
Binary variables are encoded in quantum operators acting on the qubit by simply substituting x_j -> (1-Z_j)/2 where Z_j is the Pauli Z operator acting on the j-th qubit. 

Using such cost hamiltonian we run Adiabatic Quantum Optimization and QAOA.

For the QAOA, we first attempted to use standard QOAO - 
that is we simply used sum((X(i)) for i in range(G.number_of_nodes()^2)) as mixing hamiltonian.
The algorithm actually manages to find the right solution in most of the 4000 shots - but not always, as seen in histogram.

To attempt to improve fidelity we went on implementing "S+S- mixers", again following what outiled in the paper.
Starting from an initial state in the feasible subspace (appropriately prepared), adopting this new mixing hamiltonian should in principle allows to explore all and only the states in the feasible subspace.
Unfortunately, in the end we did not manage to make this new approach work however.

On the other hand, already using a linear scheduler, the adiabatic quantum optimization proves to be quite successful in finding the system's ground state.

### 5 Nodes Graph

To figure out a 6-qubit encoding for the new 5 nodes graph, we actually started working from same nxn encoding scheme used for the previous task.
For such encoding to work we know we would in theory need 5x5 = 25 qubits.
However, it is possible to reduce the number of required qubits by exploiting some of the constraints built-in in the problem specifications. 

In particular, since initial and final points are fixed the first and last columns and rows of the 5x5 matrix are fixed.
This leaves us with a 3x3 matrix, which would need 9 qubits.
Then, if we also require to obtain a feasible solution, once the first two columns of this 3x3 matrix are set, the last column is also fixed (to satisfy the requirement that there is excatly one 1 in each row and column).
we can thus restrict ourselves to consider only the first two columns of the 3x3 encoding matrix, which we can specify with 6 qubits.

For a general graph with n nodes - with specified starting and end point - applying this encoding requires (n-2)x(n-3) qubits, so the encoding actually still scales like n^2 and we haven't gained much in terms of scaling.
However, for the specific case of a 5-node graph this encoding allows us to satisfy the 6 qubit requirement and has the advantage of using a cost hamiltonian which is a relatively simple generalization of the one used with the nxn encoding.

Numeric diagonalization of the cost hamiltonian shows that the ground state and the first excited states actually correspond to the shortest, second shortest, etc. paths.
We then went on applying both QAOA and adiabatic optimization to the cost hamiltonian to see if we could recover the expected ground state

Running adiabatic optimization using the cost hamiltonian succeeded.

QAOA on the other hand, does not work as fine.
Dumb QAOA does not work as fine (althogh we get nonzero counts for 
Since constraints are not built in we tried to increase weights of hard contraints (with respect to optimization part).
This actually had the effect of reducing the occurence of unfeasible solutions.